In [1]:
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import torch
import cupy as np
import numba 

/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:57: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:39: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:76: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


In [2]:
I=np.eye(2)
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])

In [3]:
L=8 #use even numbers
J=1
h=1.5
tol=1e-5
delta_t=.08
final_t=3.2

a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)

In [4]:

def func(gpu=True):
    h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
    h1=h*(np.kron(X,I)-np.kron(I,X))
    local_ham=qtn.LocalHam1D(L, h2, h1, cyclic=False)
    mps_sites=[np.eye(2).reshape(4)]*L
    mps_sites[int(L/2)]=np.array([[1,0],[0,-1]]).reshape(4)
    initial_state=qtn.MPS_product_state(mps_sites)
    if gpu:
        qtn.set_contract_backend('torch')
        qtn.set_tensor_linop_backend('torch')
        initial_state.apply_to_arrays(lambda x: torch.tensor(x,dtype=torch.complex128))
        local_ham.apply_to_arrays(lambda x: torch.tensor(x,dtype=torch.complex128))
    else:
        qtn.set_tensor_linop_backend('numpy')
        qtn.set_contract_backend('numpy')
        initial_state.apply_to_arrays(lambda x: np.array(x,dtype=np.complex128))
        local_ham.apply_to_arrays(lambda x: np.array(x,dtype=np.complex128))
    tebd=qtn.TEBD(initial_state,local_ham,)
    tebd.update_to(final_t,tol=tol,progbar=False,)
    return tebd.pt

In [5]:
a=func()

In [6]:
func(False)

TypeError: unhashable type: 'ndarray'